## 1-2-1. 기본 LLM 체인 (Prompt + LLM)


In [ ]:
https://wikidocs.net/book/14473
https://www.inflearn.com/courses/lecture?courseId=333326&type=LECTURE&unitId=212606


In [1]:
import langchain
print(f"LangChain 버전: {langchain.__version__}")


LangChain 버전: 1.2.3


In [3]:
from langchain_openai import ChatOpenAI

# model 
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 실행
llm.invoke("지구의 자전 주기는?").content

"지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 지구가 한 번 자전을 하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 그러나 지구는 태양 주위를 공전하고 있기 때문에, 태양 기준으로 한 자전 주기인 '태양일'은 약 24시간이 됩니다. 이처럼 태양일과 항성일의 차이는 공전 운동 때문입니다."

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser # | : 파이프 연산자 

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

"지구의 자전 주기는 약 24시간입니다. 그러나 정확히 말하면, 지구가 자신의 축을 한 바퀴 도는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 하지만 일상적으로 우리가 사용하는 24시간은 '태양일'을 기준으로 한 것으로, 태양이 같은 위치에 돌아오는데 걸리는 시간을 기준으로 합니다. 따라서 지구의 자전 주기는 대략 24시간으로 이해할 수 있습니다."

## 1-2-2. 멀티 체인 (Multi-Chain)

[text](https://wikidocs.net/231186)

멀티 체인(Multi-Chain)은 여러 개의 체인을 연결하거나 병렬로 실행하여 복잡한 워크플로우를 구성하는 패턴입니다. LCEL(LangChain Expression Language)을 사용하면 체인을 파이프(|)와 RunnableParallel로 유연하게 조합할 수 있습니다.

순차적 체인 연결: 파이프 연산자로 체인 연결
병렬 체인 실행: RunnableParallel로 동시 실행
조건부 분기: 입력에 따른 체인 선택
체인 간 데이터 전달: 출력을 다음 체인의 입력으

In [ ]:
from langchain.chat_models import init_chat_model # 다양한 모델 제공자(OpenAI, Anthropic, Google 등)의 채팅 모델을 통합된 방식으로 초기화합니다.
from langchain_core.prompts import ChatPromptTemplate # 사용자의 입력값(Variable)을 받아 미리 정의된 템플릿에 주입하여 모델에 보낼 최종 프롬프트를 만듭니다.
from langchain_core.output_parsers import StrOutputParser # 모델이 반환하는 복잡한 객체(AIMessage)에서 텍스트 내용(Content)만 추출하여 문자열로 변환합니다.
llm = init_chat_model("gpt-4o-mini")
# llm = init_chat_model("gemini-2.5-flash-lite", model_provider="google_genai")

# 1단계: 한국어를 영어로 번역
translate_prompt = ChatPromptTemplate.from_template(
    "다음 한국어를 영어로 번역하세요: {korean_word}"
)

# 2단계: 영어 단어 설명
explain_prompt = ChatPromptTemplate.from_template(
    "다음 영어 단어를 한국어로 자세히 설명하세요: {english_word}"
)

# 체인 1: 번역
chain1 = translate_prompt | llm | StrOutputParser()

# 체인 2: 번역 결과를 입력으로 사용
chain2 = (
    {"english_word": chain1}
    | explain_prompt
    | llm
    | StrOutputParser()
)

# 실행
result = chain2.invoke({"korean_word": "인공지능"})
print(result)


"인공지능"은 영어로 "artificial intelligence"라고 번역됩니다. 인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습하고, 문제를 해결하며, 의사 결정을 할 수 있도록 하는 기술을 의미합니다. 이는 기계가 데이터를 수집하고 분석하여 경험을 통해 스스로 개선할 수 있는 능력을 포함합니다. 인공지능은 자연어 처리, 이미지 인식, 자율주행차 등 다양한 분야에서 활용되고 있으며, 인간의 작업과 생활을 보다 효율적으로 변화시키는 데 기여하고 있습니다. 

인공지능은 크게 두 가지 범주로 나눌 수 있습니다. 첫째는 좁은 인공지능(weak AI)으로, 특정 작업을 수행하는 데 특화된 시스템을 의미합니다. 예를 들어, 추천 알고리즘이나 음성 인식 시스템은 이러한 좁은 인공지능의 예입니다. 둘째는 광범위한 인공지능(strong AI)으로, 이는 인간처럼 여러 가지 작업을 수행하고, 스스로 사고하며, 감정을 느낄 수 있는 인공지능을 의미합니다. 현재의 기술 수준에서는 아직 광범위한 인공지능에 도달하지는 못했지만, 연구는 계속 진행되고 있습니다. 

인공지능은 또한 머신러닝(기계 학습)과 딥러닝(심층 학습)과 같은 하위 분야를 포함하며, 이를 통해 컴퓨터는 데이터를 기반으로 패턴을 학습하고 미래의 예측을 수행할 수 있습니다. 이러한 기술은 여러 산업에서 혁신을 가져오고 있으며, 인공지능을 활용한 솔루션이 점점 더 많이 도입되고 있습니다.


### 다단계 순차 체인


In [1]:
from langchain.chat_models import init_chat_model # 다양한 모델 제공자(OpenAI, Anthropic, Google 등)의 채팅 모델을 통합된 방식으로 초기화합니다.
from langchain_core.prompts import ChatPromptTemplate # 사용자의 입력값(Variable)을 받아 미리 정의된 템플릿에 주입하여 모델에 보낼 최종 프롬프트를 만듭니다.
from langchain_core.output_parsers import StrOutputParser # 모델이 반환하는 복잡한 객체(AIMessage)에서 텍스트 내용(Content)만 추출하여 문자열로 변환합니다.
from langchain_core.runnables import RunnablePassthrough # 체인을 통과하는 데이터를 수정 없이 그대로 전달하거나, 특정 데이터를 추가 및 할당하는 중간 통로 역할을 합니다.

llm = init_chat_model("gpt-4o-mini")

# 3단계 처리: 주제 분석 → 개요 작성 → 본문 작성
analyze_prompt = ChatPromptTemplate.from_template(
    "다음 주제의 핵심 키워드 3개를 추출하세요: {topic}"
)

outline_prompt = ChatPromptTemplate.from_template(
    """다음 키워드를 바탕으로 글의 개요를 작성하세요:
키워드: {keywords}
원본 주제: {topic}"""
)

content_prompt = ChatPromptTemplate.from_template(
    """다음 개요를 바탕으로 300자 내외의 글을 작성하세요:
개요: {outline}"""
)

# 체인 구성
chain = (
    {"topic": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        keywords=analyze_prompt | llm | StrOutputParser()
    )
    | RunnablePassthrough.assign(
        outline=outline_prompt | llm | StrOutputParser()
    )
    | content_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("기후 변화와 지속 가능한 발전")
print(result)


/home/bigseoul/workspace/indgram/.venv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


기후 변화는 인류의 과도한 자원 소비와 온실가스 배출로 인해 발생하고 있으며, 이는 지구의 생태계와 인류 사회에 심각한 영향을 미치고 있습니다. 이러한 문제를 해결하기 위해 지속 가능한 발전이 필수적입니다. 지속 가능성은 인류의 필요와 환경 보호를 조화롭게 이루는 개념으로, 유엔의 지속 가능한 발전 목표(SDGs)로 구체화됩니다.

재생 가능 에너지는 이러한 지속 가능한 발전의 핵심 요소로, 태양광, 풍력, 수력, 바이오에너지 등 다양한 형태로 존재합니다. 이들 에너지원은 온실가스를 줄이고 에너지 자립성을 강화하는 데 기여하며, 성공적인 사례를 통해 그 효과성을 입증하고 있습니다. 기후 변화에 대응하기 위해서는 지속 가능한 발전에 대한 의식 전환이 필요하며, 특히 개인이 적극적으로 재생 가능 에너지를 지지하고 활용하는 노력이 중요합니다. 이러한 변화가 궁극적으로 건강한 지구와 지속 가능한 미래를 만들어 갈 것입니다.
